In [12]:
#| include: false
#| default_exp utils

In [13]:
#| export


import _ast
import ast
from pathlib import Path
import nbformat

from nbdev.processors import _default_exp
from execnb.nbio import read_nb
from nbdev.export import get_config

In [14]:
#| export


def load_nb(nb_path):
    nb = read_nb(nb_path)
    default_export = _default_exp(nb)
    if default_export is None:
        raise ValueError(f"{nb_path.name} does not contain an associated nbdev module")

    module_name = default_export.replace(".", "/")
    module_path = Path(get_config().path("lib_path"), f"{module_name}.py")
    return nb, module_path

In [15]:
nb, module_path = load_nb("scilint.ipynb")
assert Path(module_path).exists()

check = False
try:
    nb, module_path = load_nb(Path(Path(".").resolve(), "index.ipynb"))
except ValueError:
    check = True
assert check

In [16]:
#| export

def load_nb_module(nb_path):
    nb, module_path = load_nb(nb_path)
    with open(module_path, "r") as module_file:
        lines = module_file.readlines()
    module_code = "\n".join(lines)
    return nb, module_code

In [17]:
nb, module_code = load_nb_module(Path("scilint.ipynb"))

assert type(ast.parse(module_code)) == _ast.Module